In [1]:
from pandas import DataFrame, read_csv
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.metrics import accuracy_score
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression 
from warnings import filterwarnings as fw
fw("ignore")
#data = pd.read_csv("/home/naksh/Desktop/dataset/train/problem_data.csv")
f1 = r'/home/naksh/Desktop/dataset/train/problem_data.csv'
df = pd.read_csv(f1)
print(len(df.index))
#print(df)
f2 = r'/home/naksh/Desktop/dataset/train/train_submissions.csv'
train_sub = pd.read_csv(f2)

6544


In [2]:
df.dropna(subset=['level_type','points','tags'],how='all',inplace=True)
df.shape

(6456, 4)

In [3]:
df.dropna(axis=0,  subset=['level_type'], inplace=True)
df.shape

(6411, 4)

In [4]:
level_type={'A':5,'B':10,'C':15,'D':20,'E':25,'F':30,'G':35,'H':40,'I':45,'J':50,'K':55,'L':60,'M':65,'N':70}
new_points=[level_type.get(i,0) for i in list(df['level_type'])]
df['points']=new_points
df.head()

,problem_id,level_type,points,tags
0,prob_3649,H,40,NaN
1,prob_6191,A,5,NaN
2,prob_2020,F,30,NaN
3,prob_313,A,5,"greedy,implementation"
4,prob_101,A,5,"constructive algorithms,greedy,math"


In [5]:
df.to_csv("/home/naksh/Documents/DA/project/processed.csv", sep=',', encoding='utf-8')

In [6]:
f=r'/home/naksh/Documents/DA/project/processed.csv'
p=pd.read_csv(f)

In [7]:
import numpy as np
dis=train_sub.groupby('problem_id')['user_id'].agg(np.size).reset_index(name='counts')
#distinct=dis['counts'].str.replace(',', ".").astype(float)
#for i in dis['counts']:
 #   i = float(i)
#train_sub.head()
distinct = dis
distinct.head()

,problem_id,counts
0,prob_1,2
1,prob_10,1
2,prob_100,1
3,prob_1000,246
4,prob_1001,10


In [8]:
print(p.shape)
mer=pd.merge(p,distinct,how='left',on='problem_id')
#mer=pd.merge(p,distinct,on=['problem_id','problem_id'])
print(mer.shape)
mer=mer.drop(['Unnamed: 0'], axis=1)
mer['counts'].fillna(0,inplace=True)
print(mer.shape)
mer['counts'] = [int(i) for i in mer['counts']]
mer.head(10)

(6411, 5)
(6411, 6)
(6411, 5)


,problem_id,level_type,points,tags,counts
0,prob_3649,H,40,NaN,3
1,prob_6191,A,5,NaN,3
2,prob_2020,F,30,NaN,1
3,prob_313,A,5,"greedy,implementation",17
4,prob_101,A,5,"constructive algorithms,greedy,math",23
5,prob_1479,E,25,NaN,0
6,prob_2940,B,10,NaN,1
7,prob_3059,B,10,brute force,4
8,prob_1958,D,20,NaN,3
9,prob_6154,C,15,NaN,3


In [9]:
# level prediction using no of people SVM
train=mer.iloc[:,:]
test=mer.iloc[:,:]
x_train = train.iloc[:,4]
y_train = train.iloc[:,1]
x_train=[[i] for i in x_train]
x_test = test.iloc[:,4]
y_test = test.iloc[:,1]
x_test=[[i] for i in x_test]
clf = svm.SVC(decision_function_shape='ovo', kernel='linear')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 0.16877242239900173
Confusion matrix: 
 [[ 91 353   0   0 598   0   0   0   0   0   0   0   0   0]
 [ 62 279   0   0 676   0   0   0   0   0   0   0   0   0]
 [ 41 207   0   0 667   0   0   0   0   0   0   0   0   0]
 [  7 168   0   0 675   0   0   0   0   0   0   0   0   0]
 [  1  82   0   0 712   0   0   0   0   0   0   0   0   0]
 [  0  35   0   0 386   0   0   0   0   0   0   0   0   0]
 [  0  15   0   0 313   0   0   0   0   0   0   0   0   0]
 [  0  15   0   0 257   0   0   0   0   0   0   0   0   0]
 [  0  12   0   0 244   0   0   0   0   0   0   0   0   0]
 [  0  10   0   0 202   0   0   0   0   0   0   0   0   0]
 [  0   8   0   0 146   0   0   0   0   0   0   0   0   0]
 [  0   3   0   0  96   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0  38   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0   8   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.45      0.09      0.15      1042
          B      

In [10]:
# level prediction using no of people LOGISTIC
#train=mer.iloc[:3206,:]
#test=mer.iloc[3206:,:]
x_train = mer[['counts']]
y_train =mer['level_type']
x_test = mer.iloc[:,4]
y_test = mer.iloc[:,1]
x_test=[[i] for i in x_test]
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))


Accuracy 0.1734518795819685
Confusion matrix: 
 [[380 662   0   0   0   0   0   0   0   0   0   0   0   0]
 [285 732   0   0   0   0   0   0   0   0   0   0   0   0]
 [204 711   0   0   0   0   0   0   0   0   0   0   0   0]
 [123 727   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 59 736   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 22 399   0   0   0   0   0   0   0   0   0   0   0   0]
 [  9 319   0   0   0   0   0   0   0   0   0   0   0   0]
 [  5 267   0   0   0   0   0   0   0   0   0   0   0   0]
 [  4 252   0   0   0   0   0   0   0   0   0   0   0   0]
 [  5 207   0   0   0   0   0   0   0   0   0   0   0   0]
 [  3 151   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  99   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1  39   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  10   0   0   0   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.35      0.36      0.35      1042
          B       

In [11]:
tags = set()
for i in [i.split(',') for i in list(set(mer['tags']))[1:]]:
    for j in i:
        tags.add(j)
tags = list(tags)
print(len(tags))

36


In [12]:
bow = []
for i in mer['tags']:
    if isinstance(i, float):
        bow.append([0]*len(tags))
        continue
    new = []
    for j in tags:
        new.append(1 if j in i.split(',') else 0)
    bow.append(new)
bow = list(map(list, zip(*bow)))


In [13]:
#adding columns of bag of words to df
for ii, i in enumerate(tags):
    mer[i] = bow[ii]

In [14]:
temp = [ord(i)-65 if not isinstance(i, float) else 3 for i in mer['level_type']]
mer['level_encoded'] = temp

In [15]:
mer.shape

#mer.head(10)

(6411, 42)

In [16]:
x_train = mer[['counts','level_encoded']]
y_train =mer['level_type']
x_test =mer[['counts','level_encoded']]
y_test =mer['level_type']

lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))


Accuracy 0.6663547028544688
Confusion matrix: 
 [[1042    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0 1017    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0   56  859    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0  150  700    0    0    0    0    0    0    0    0    0    0]
 [   0    0    3  143  649    0    0    0    0    0    0    0    0    0]
 [   0    0    0   28  393    0    0    0    0    0    0    0    0    0]
 [   0    0    0    7  321    0    0    0    0    0    0    0    0    0]
 [   0    0    0    1  271    0    0    0    0    0    0    0    0    0]
 [   0    0    0    1  255    0    0    0    0    0    0    0    0    0]
 [   0    0    0    1   41    0    0    0  166    4    0    0    0    0]
 [   0    0    0    0    0    0    0    0   73   80    1    0    0    0]
 [   0    0    0    0    0    0    0    0    0   99    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0   40    0    0    0]
 [ 

In [17]:
df[:4].head()

,problem_id,level_type,points,tags
0,prob_3649,H,40,NaN
1,prob_6191,A,5,NaN
2,prob_2020,F,30,NaN
3,prob_313,A,5,"greedy,implementation"


In [18]:
mer.head()
#mer.shape

,problem_id,level_type,points,tags,counts,number theory,flows,shortest paths,brute force,games,...,dsu,schedules,hashing,combinatorics,trees,strings,greedy,2-sat,meet-in-the-middle,level_encoded
0,prob_3649,H,40,NaN,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,prob_6191,A,5,NaN,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,prob_2020,F,30,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,prob_313,A,5,"greedy,implementation",17,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,prob_101,A,5,"constructive algorithms,greedy,math",23,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
# level prediction using no of people and tags LOGISTIC
'''
split=int(mer.shape[0]*0.9)
train=mer.iloc[:split,:]
test=mer.iloc[split:,:]
x_train= train.iloc[:,4:40]
y_train = train.iloc[:,1]

x_test = test.iloc[:,4:40]
y_test = test.iloc[:,1]
#x_train=[[i] for i in x_train]
#print(x_train.shape) %%!'''
x_train= mer.iloc[:,4:40]
y_train = mer.iloc[:,1]
#x_train=[[i] for i in x_train]
x_test = mer.iloc[:,4:40]
y_test = mer.iloc[:,1]
#print(x_train)
#x_test=[[i] for i in x_test]
clf = LogisticRegression(multi_class='multinomial',solver ='newton-cg')

clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracy", round(accuracy_score(y_test, y_pred)*100,3))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 24.099
Confusion matrix: 
 [[854  89  62  16  12   9   0   0   0   0   0   0   0   0]
 [625 171 107  64  44   5   1   0   0   0   0   0   0   0]
 [449 123 146  79 106  12   0   0   0   0   0   0   0   0]
 [399  68  98 113 165   6   1   0   0   0   0   0   0   0]
 [364  30  58  81 252  10   0   0   0   0   0   0   0   0]
 [316   9  18  19  50   9   0   0   0   0   0   0   0   0]
 [290   2   5   5  19   7   0   0   0   0   0   0   0   0]
 [256   2   2   5   3   4   0   0   0   0   0   0   0   0]
 [246   1   2   1   4   2   0   0   0   0   0   0   0   0]
 [205   2   2   1   1   1   0   0   0   0   0   0   0   0]
 [153   0   0   1   0   0   0   0   0   0   0   0   0   0]
 [ 96   0   1   1   1   0   0   0   0   0   0   0   0   0]
 [ 37   3   0   0   0   0   0   0   0   0   0   0   0   0]
 [  9   0   0   0   1   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.20      0.82      0.32      1042
          B       0.34      0.

In [20]:
### level prediction using no of people BEROULLI
x_train = mer.iloc[:,4]
#print("x_train")
#print(x_train)
y_train = mer.iloc[:,1]
#print("y_train")
#print(y_train)
x_test = mer.iloc[:,4]
y_test = mer.iloc[:,1]
x_train=[[i] for i in x_train]

x_test=[[i] for i in x_test]
clf = BernoulliNB()
#print(mer.iloc[:,5:])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 0.1673685852441117
Confusion matrix: 
 [[969   0   0   0  73   0   0   0   0   0   0   0   0   0]
 [931   0   0   0  86   0   0   0   0   0   0   0   0   0]
 [835   0   0   0  80   0   0   0   0   0   0   0   0   0]
 [750   0   0   0 100   0   0   0   0   0   0   0   0   0]
 [691   0   0   0 104   0   0   0   0   0   0   0   0   0]
 [354   0   0   0  67   0   0   0   0   0   0   0   0   0]
 [278   0   0   0  50   0   0   0   0   0   0   0   0   0]
 [226   0   0   0  46   0   0   0   0   0   0   0   0   0]
 [198   0   0   0  58   0   0   0   0   0   0   0   0   0]
 [168   0   0   0  44   0   0   0   0   0   0   0   0   0]
 [136   0   0   0  18   0   0   0   0   0   0   0   0   0]
 [ 90   0   0   0   9   0   0   0   0   0   0   0   0   0]
 [ 34   0   0   0   6   0   0   0   0   0   0   0   0   0]
 [  7   0   0   0   3   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.17      0.93      0.29      1042
          B       

In [21]:
### level prediction using no of people and tags K NEAREST NEIGHBOURS
scaler = StandardScaler()

x_train = mer.iloc[:,4:-1]
scaler.fit(x_train)
x_train = scaler.transform(x_train)
y_train=mer.iloc[:,1]

x_test = mer.iloc[:,4:-1]
scaler.fit(x_test)
x_test = scaler.transform(x_test) 
y_test=mer.iloc[:,1]

classifier = KNeighborsClassifier(n_neighbors=1) #neighbours 5 accuracy=30.86 
classifier.fit(x_train, y_train)  
y_pred = classifier.predict(x_test) 
print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 0.45671502105755735
Confusion matrix: 
 [[673  66  18  17  21 114   7  63  27   7  27   2   0   0]
 [171 577  22  12  20  96   3  66  22  10  16   2   0   0]
 [111  30 497  24  27 111   2  66  20   6  18   3   0   0]
 [110  17  21 429  46 103   1  72  21   6  22   2   0   0]
 [111  17  21  19 418 109   5  55  11   7  19   2   1   0]
 [110   6   2  16  15 178   1  61  15   3  13   1   0   0]
 [ 99   3   1   6   9  82  36  48  17   4  22   1   0   0]
 [ 88   2   0   5   8  66   1  68  12   5  15   2   0   0]
 [ 80   5   0   4  14  64   0  57  18   1  13   0   0   0]
 [ 65   4   0   8   1  47   0  49   7  15  12   4   0   0]
 [ 46   4   0   7   8  38   0  20  17   2  11   1   0   0]
 [ 34   4   0   1   5  17   0  10  15   3   5   5   0   0]
 [ 15   1   0   1   3   9   0   6   0   2   1   0   2   0]
 [  0   0   0   0   2   3   0   3   0   0   1   0   0   1]]
             precision    recall  f1-score   support

          A       0.39      0.65      0.49      1042
          B      

In [22]:
### level prediction using no of people and tags RANDOM FOREST
clf=RandomForestClassifier(n_estimators=100)
x_train = mer.iloc[:,4:-1]
y_train=mer.iloc[:,1]

x_test = mer.iloc[:,4:-1]
y_test=mer.iloc[:,1]

clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)

print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))



Accuracy 0.48479176415535796
Confusion matrix: 
 [[828  67  17  86  28  11   1   0   0   4   0   0   0   0]
 [250 608  19  91  36   8   0   0   0   5   0   0   0   0]
 [197  58 505 102  31  14   1   0   0   7   0   0   0   0]
 [189  43  17 534  45  13   0   3   0   5   1   0   0   0]
 [189  38  18  87 443   9   1   1   0   5   2   0   1   1]
 [159  36   9  74  30 106   1   1   0   3   2   0   0   0]
 [169  34   4  57  26   5  29   0   0   4   0   0   0   0]
 [132  35   5  53  17   4   0  19   0   6   1   0   0   0]
 [123  28   6  64  19   5   0   1   6   2   2   0   0   0]
 [ 98  22   3  49  13   8   0   0   0  19   0   0   0   0]
 [ 69  24   2  32  10   7   0   0   0   4   6   0   0   0]
 [ 50  13   1  20   8   1   0   0   0   3   2   1   0   0]
 [ 22   3   0   7   5   1   0   0   0   0   0   0   2   0]
 [  4   0   0   3   0   0   0   0   0   0   1   0   0   2]]
             precision    recall  f1-score   support

          A       0.33      0.79      0.47      1042
          B      

In [23]:
poly = PolynomialFeatures(degree= 2)
x_train = mer.iloc[:,4:-1]
y_train=mer.iloc[:,1]

x_test = mer.iloc[:,4:-1]
y_test=mer.iloc[:,1]

#x_train=[[i] for i in x_train]
#x_test=[[i] for i in x_test]

x_poly = poly.fit_transform(x_train) 
#y_poly=poly.fit_transform(y_train)
#T2 = [float[i] for i in x_poly]

x_poly_test = poly.fit_transform(x_test) 

poly.fit(x_poly, y_train) 
lin2 = LinearRegression() 
lin2.fit(x_poly, y_train)
y_pred=lin2.predict(x_poly_test)

print("Accuracy", accuracy_score(y_test, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))



ValueError: could not convert string to float: 'B'